# PANGAEApy
## Introduction and examples for PANGAEA community workshop 2022

### This script shows an example of searching and downloading multiple files via pangaeapy and how to get information on meta data

#### For more information on pangaeapy: https://github.com/pangaea-data-publisher/pangaeapy


### Overview
* How to search for specific data sets in PANGAEA
* Search by project
* Convert search results into one easy to read table
* Export search results to csv file
* Get data including meta data of a single data set
* Refine search with geographical coordinates by applying a bounding box
* Refine search further: filter only datasets with "Geochemistry" in title
* Get multiple data sets
* Download many binary files
* Download many files

In [1]:
# import necessary packages
import pangaeapy as pan
from pangaeapy.pandataset import PanDataSet
import pandas as pd
import re
import os

In [2]:
# show functions of pangaeapy
help(pan)

Help on package pangaeapy:

NAME
    pangaeapy

DESCRIPTION
    pangaeapy is a package allowing to download and analyse metadata
    as well as data from tabular PANGAEA (https://www.pangaea.de) datasets.

PACKAGE CONTENTS
    exporter (package)
    mappings (package)
    pandataset
    panquery

FILE
    c:\users\kriemann\anaconda3\lib\site-packages\pangaeapy\__init__.py




# How to search for specific data sets in PANGAEA

In [3]:
# show functions of pangaeapy.panquery
help(pan.panquery)

Help on module pangaeapy.panquery in pangaeapy:

NAME
    pangaeapy.panquery

CLASSES
    builtins.object
        PanQuery
    
    class PanQuery(builtins.object)
     |  PanQuery(query, bbox=(), limit=10, offset=0)
     |  
     |  PANGAEA Query Class
     |  This class allows to query PANGAEA and to access query results
     |  
     |  Parameters
     |  ----------
     |  query : str
     |      The query string following the specs at www.pangaea.de
     |  bbox : set
     |      The bounding box to define  geographical search constraints following the GeoJSON specs
     |      example : bbox=(minlon, minlat,  maxlon, maxlat)
     |  limit : int
     |      The expected number of search results (max =100)
     |  offset : int
     |      The offset of the search to continue results retrieval
     |  
     |  Attributes
     |  ----------
     |  totalcount : int
     |      The number of total search results
     |  error : str
     |      In case an error occurs this attribute ho

##  Search by project
### Example: search for project "PAGES_C-PEAT"
pan.PanQuery("PAGES_C-PEAT", limit = 500)
vs. 
pan.PanQuery("project:label:PAGES_C-PEAT", limit = 500)


Note: the default limit = 10, maximum of limit = 500

In [4]:
search1 = pan.PanQuery("PAGES_C-PEAT", limit = 500)
print(search1.totalcount)
print(search1.error)
print(search1.query)

876
None
PAGES_C-PEAT


In [5]:
#search1.result

#### Documentation on search with keywords
https://wiki.pangaea.de/wiki/PANGAEA_search

In [6]:
# refined search with project label
# same as search on website https://www.pangaea.de/?q=project:label:PAGES_C-PEAT
search2 = pan.PanQuery("project:label:PAGES_C-PEAT", limit = 500)
print(search2.totalcount)
print(search2.error)
print(search2.query)

875
None
project:label:PAGES_C-PEAT


In [7]:
search2.result

[{'URI': 'doi:10.1594/PANGAEA.928076',
  'score': 6.438728,
  'html': '<li><div class="citation"><a href="https://doi.pangaea.de/10.1594/PANGAEA.928076" target="_self" class="dataset-link"><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Geochemistry of Sebangau swamp forest (SEB 5A) peat core from Borneo</a></div><table class="result" summary="Dataset reference and size" cellspacing="0" cellpadding="0"><tr><td class="title">Related to:</td><td class="content"><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Age determination of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Calibrated ages with Pb210 of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Calibrated ages without Pb210 of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><small><em>(and more)</em></small></td></tr><tr><td class="title">Size:</td><td class="content">840 data p

#### Hint: specify your search with facet filter at https://www.pangaea.de and refine your search query with PANGAEApy

### What if list of search results exceeds limit of 500?
If search has a result list (totalcount) > 500, split search results in 2 querys

In [8]:
PAGES1=pan.PanQuery("project:label:PAGES_C-PEAT", limit = 500)
print(PAGES1.totalcount)
print(PAGES1.error)
print(PAGES1.query)
print(PAGES1.result[0]['URI'])

875
None
project:label:PAGES_C-PEAT
doi:10.1594/PANGAEA.928076


In [9]:
PAGES2=pan.PanQuery("project:label:PAGES_C-PEAT", limit = 500, offset=500)
print(PAGES2.totalcount)
print(PAGES2.error)
print(PAGES2.query)
print(PAGES2.result[0]['URI'])

875
None
project:label:PAGES_C-PEAT
doi:10.1594/PANGAEA.929647


In [10]:
print(type(PAGES1))
print(type(PAGES1.result))
print(PAGES1.result[0])

<class 'pangaeapy.panquery.PanQuery'>
<class 'list'>
{'URI': 'doi:10.1594/PANGAEA.928076', 'score': 6.438728, 'html': '<li><div class="citation"><a href="https://doi.pangaea.de/10.1594/PANGAEA.928076" target="_self" class="dataset-link"><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Geochemistry of Sebangau swamp forest (SEB 5A) peat core from Borneo</a></div><table class="result" summary="Dataset reference and size" cellspacing="0" cellpadding="0"><tr><td class="title">Related to:</td><td class="content"><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Age determination of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Calibrated ages with Pb210 of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Calibrated ages without Pb210 of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><small><em>(and more)</em></small></td></tr><tr><td class=

### Convert search results into one easy to read table
convert list of dictionaries into data frame 

In [11]:
df1 = pd.DataFrame(PAGES1.result)
df2 = pd.DataFrame(PAGES2.result)

information on size of data frames and columns

In [12]:
df1.columns

Index(['URI', 'score', 'html', 'type', 'position'], dtype='object')

In [13]:
df1.count()

URI         500
score       500
html        500
type        500
position    500
dtype: int64

In [14]:
df2.count()

URI         375
score       375
html        375
type        375
position    375
dtype: int64

merge both data frames into one

In [15]:
df=pd.concat([df1,df2],ignore_index=True)
df.count()

URI         875
score       875
html        875
type        875
position    875
dtype: int64

show first 5 lines of data frame

In [16]:
df.head()

,URI,score,html,type,position
0,doi:10.1594/PANGAEA.928076,6.438728,"<li><div class=""citation""><a href=""https://doi...",child,0
1,doi:10.1594/PANGAEA.927702,6.438728,"<li><div class=""citation""><a href=""https://doi...",child,1
2,doi:10.1594/PANGAEA.890510,6.438728,"<li><div class=""citation""><a href=""https://doi...",child,2
3,doi:10.1594/PANGAEA.927692,6.438728,"<li><div class=""citation""><a href=""https://doi...",child,3
4,doi:10.1594/PANGAEA.927414,6.438728,"<li><div class=""citation""><a href=""https://doi...",child,4


which information is in column html?

In [17]:
df.html[0]

'<li><div class="citation"><a href="https://doi.pangaea.de/10.1594/PANGAEA.928076" target="_self" class="dataset-link"><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Geochemistry of Sebangau swamp forest (SEB 5A) peat core from Borneo</a></div><table class="result" summary="Dataset reference and size" cellspacing="0" cellpadding="0"><tr><td class="title">Related to:</td><td class="content"><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Age determination of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Calibrated ages with Pb210 of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><strong>Charman, DJ; Gallego-Sala, AV (2021):</strong> Calibrated ages without Pb210 of Sebangau swamp forest (SEB 5A) peat core from Borneo.<br/><small><em>(and more)</em></small></td></tr><tr><td class="title">Size:</td><td class="content">840 data points</td></tr></table><div class="datasetid"><a href="https://doi.pan

#### get information on titel and author out of html code and add to data frame
use regular expressions

In [18]:
# create column: titel
df['titel'] = df.html.str.extract(r'"citation">(?:.*?)<\/strong>(.*?)<\/a>')

# create column: author
df['author(s)'] = df.html.str.extract(r'"dataset-link"><strong>(.*?)\([0-9]{4}\):<\/strong>')

# create column: year of publication
df['year of publication'] = df.html.str.extract(r'(\([0-9]{4}\))')

#create column: PANGAEA ID
df['PANGAEA ID'] = df.html.str.extract(r'class="citation"><a href="https:\/\/doi.pangaea.de\/10.1594\/PANGAEA.([0-9]{6})')

#print(df.columns)
# adapt position of columns 
df = df[['PANGAEA ID','author(s)', 'titel','year of publication','URI','type','score','position', 'html']]

In [19]:
df

,PANGAEA ID,author(s),titel,year of publication,URI,type,score,position,html
0,928076,"Charman, DJ; Gallego-Sala, AV",Geochemistry of Sebangau swamp forest (SEB 5A...,(2021),doi:10.1594/PANGAEA.928076,child,6.438728,0,"<li><div class=""citation""><a href=""https://doi..."
1,927702,"Martínez Cortizas, A",Geochemistry of Penido Vello (PVO) peat core ...,(2021),doi:10.1594/PANGAEA.927702,child,6.438728,1,"<li><div class=""citation""><a href=""https://doi..."
2,890510,"Charman, DJ",Age determination of Sidney_core1 peat core,(2018),doi:10.1594/PANGAEA.890510,child,6.438728,2,"<li><div class=""citation""><a href=""https://doi..."
3,927692,"Martínez Cortizas, A",Geochemistry of Borralleiras da Cal Grande (B...,(2021),doi:10.1594/PANGAEA.927692,child,6.438728,3,"<li><div class=""citation""><a href=""https://doi..."
4,927414,"Moss, PT",Geochemistry of Moon Point Patterned Fen MP p...,(2021),doi:10.1594/PANGAEA.927414,child,6.438728,4,"<li><div class=""citation""><a href=""https://doi..."
...,...,...,...,...,...,...,...,...,...
870,929867,"Yu, Z",Calibrated ages of No Name Creek (NNC) peat c...,(2021),doi:10.1594/PANGAEA.929867,child,6.156247,870,"<li><div class=""citation""><a href=""https://doi..."
871,941140,"Turner, TE",Age determination of Malham Tarn Moss (MTM) p...,(2022),doi:10.1594/PANGAEA.941140,child,6.156247,871,"<li><div class=""citation""><a href=""https://doi..."
872,934124,"Zhao, Y",Age determination of Altay peat core from China,(2021),doi:10.1594/PANGAEA.934124,child,6.156247,872,"<li><div class=""citation""><a href=""https://doi..."
873,935018,"van Bellen, S",Age determination of Mosaik L1 (MOS_L1) peat ...,(2021),doi:10.1594/PANGAEA.935018,child,6.156247,873,"<li><div class=""citation""><a href=""https://doi..."


type = child means, that data set is part of a data collection

score indicates how well the data set matches the search query

## Export search results to csv file

Find out what your current path is and alter it to your liking.

In [20]:
# what is my Current Working Directory ?
print(os.getcwd())

C:\Users\kriemann\python\pangaeapy


Define the path and file name where the output will be stored.

##### NOTE: If you are working on a Windows machine: \ need to be / and don't forget the last /

In [21]:
datapath='C:/Users/kriemann/python/pangaeapy/'
outfile='search_result_PAGES.txt'

Export list in csv formatted file

In [22]:
df.to_csv((datapath+outfile),sep='\t',index=False)

## Get data including meta data of a single data set
#### use function PanDataSet

In [23]:
#help(PanDataSet)

In [24]:
Joey_core12 = PanDataSet(890405)
print(Joey_core12.title)
print(Joey_core12.citation)
print(Joey_core12.isParent)

Geochemistry of Joey_core12 peat core
Camill, Philip (2018): Geochemistry of Joey_core12 peat core. PANGAEA, https://doi.org/10.1594/PANGAEA.890405
False


In [25]:
Joey_core12.data.head()

,Depth,Age,DBD,TC,TN,Corg dens,Event,Latitude,Longitude,Elevation
0,0.01,0.007,0.034,47.0,0.6,0.016,Joey_core12,55.47,-98.15,214.0
1,0.03,0.022,0.044,48.0,0.5,0.021,Joey_core12,55.47,-98.15,214.0
2,0.05,0.037,0.047,48.9,0.6,0.023,Joey_core12,55.47,-98.15,214.0
3,0.07,0.052,0.048,48.9,0.6,0.023,Joey_core12,55.47,-98.15,214.0
4,0.09,0.066,0.049,49.9,0.7,0.025,Joey_core12,55.47,-98.15,214.0


Parameter long names and units are given in lists 

In [26]:
long_names = []
for param in Joey_core12.params.values():
    print(param.name)
    print(param.unit)
    long_names.append(str(param.name) + ' [' + str(param.unit) + ']')
    
#print(long_names)

DEPTH, sediment/rock
m
AGE
ka BP
Density, dry bulk
g/cm**3
Carbon, total
%
Nitrogen, total
%
Density, organic carbon
g/cm**3
Event
None
Latitude
deg
Longitude
deg
Elevation
m


### download table as tab-delimited txt file

In [27]:
datapath='C:/Users/kriemann/python/pangaeapy/'
outfile_joey='Joey_core12.txt'

Joey_core12.data.to_csv((datapath+outfile_joey),sep='\t',index=False,header=long_names)


## Refine search with geographical coordinates by applying a bounding box

bbox: set the bounding box to define geographical search constraints following the GeoJSON specs


bbox=(minlon, minlat, maxlon, maxlat)

In [28]:
# datasets in northern Sweden
PAGES_Sweden = pan.PanQuery("project:label:PAGES_C-PEAT", limit = 500, bbox=(17.7, 67.7, 21, 69))
print(PAGES_Sweden.totalcount)
print(PAGES_Sweden.error)
print(PAGES_Sweden.query)


25
None
project:label:PAGES_C-PEAT


loop over result list and take PANGAEA data set ID from URI

In [29]:
panID = []
title = []
for count,value in enumerate(PAGES_Sweden.result):
    #print(PAGES_Sweden.result[count]['URI'].split('.'))
    c = []
    a,b,c = PAGES_Sweden.result[count]['URI'].split('.')
    #print(int(c))
    panID.append(int(c))
    
    df_tmp = PanDataSet(int(c))
    #print(df_tmp.title)
    title.append(df_tmp.title)
    
    
#print(panID)
#print(title)

In [30]:
df_sweden_meta = pd.DataFrame(panID,columns=['panID'])
df_sweden_meta['Title'] = title

In [31]:
df_sweden_meta.head()

,panID,Title
0,929541,Calibrated ages of Stordalen 2 peat core from ...
1,890490,Geochemistry of Stordalen peat core
2,929542,Calibrated ages of Stordalen 3 (ST-dessicating...
3,927837,Geochemistry of Instrument Bog (I1-6) peat cor...
4,927843,Geochemistry of Railway Bog (R1-7) peat core f...


In [32]:
df_sweden_meta.count()

panID    25
Title    25
dtype: int64

## Refine search further: filter only datasets with "Geochemistry" in title

In [33]:
PAGES_Sweden_geo = df_sweden_meta[df_sweden_meta['Title'].str.contains('Geochemistry')]

In [34]:
PAGES_Sweden_geo.head()

,panID,Title
1,890490,Geochemistry of Stordalen peat core
3,927837,Geochemistry of Instrument Bog (I1-6) peat cor...
4,927843,Geochemistry of Railway Bog (R1-7) peat core f...
5,927841,Geochemistry of Nikka (N1-6) peat core from Ab...
14,927840,Geochemistry of Marooned (M1-7) peat core from...


In [35]:
PAGES_Sweden_geo.count()

panID    11
Title    11
dtype: int64

## Get multiple data sets

combine all data of PAGES_Sweden_geo search results into a single data frame


In [36]:
# new data frame
PAGES_Sweden_data = []
first = True

for i, id_pan in PAGES_Sweden_geo['panID'].iteritems():
    #print(i)
    #print(id_pan)
    df_tmp = []
    df_tmp = PanDataSet(id_pan)
    df_tmp.data['DOI'] = df_tmp.doi
    df_tmp.data['citation'] = df_tmp.citation
    
    if first == True:
        PAGES_Sweden_data = pd.DataFrame(df_tmp.data)
        first = False
    else:
        PAGES_Sweden_data = pd.concat([PAGES_Sweden_data,df_tmp.data], axis=0, ignore_index=True)


In [37]:
# rearange order of columns
print(PAGES_Sweden_data.columns)
PAGES_Sweden_data = PAGES_Sweden_data[['Depth', 'Age', 'DBD', 'OM', 'OM dens', 'TC', 'TN', 'Corg dens', 'Peat',
       'Peat_2', 'Samp thick', 'LOI', 'C', 'Event', 'Latitude', 'Longitude', 'Elevation', 'DOI', 'citation']]

Index(['Depth', 'Age', 'DBD', 'OM', 'OM dens', 'TC', 'TN', 'Corg dens', 'Peat',
       'Peat_2', 'Event', 'Latitude', 'Longitude', 'Elevation', 'DOI',
       'citation', 'Samp thick', 'LOI', 'C'],
      dtype='object')


In [38]:
PAGES_Sweden_data.count()

Depth         843
Age           115
DBD           843
OM            222
OM dens       222
TC            811
TN            811
Corg dens     218
Peat          225
Peat_2        105
Samp thick    618
LOI           102
C             102
Event         843
Latitude      843
Longitude     843
Elevation     843
DOI           843
citation      843
dtype: int64

In [39]:
PAGES_Sweden_data.head()

,Depth,Age,DBD,OM,OM dens,TC,TN,Corg dens,Peat,Peat_2,Samp thick,LOI,C,Event,Latitude,Longitude,Elevation,DOI,citation
0,0.003,0.000,0.097,98.2,0.095,45.8,0.8,0.044,Dicranum elongatum peat,5.0,NaN,NaN,NaN,Stordalen_1,68.35,19.05,350.0,https://doi.org/10.1594/PANGAEA.890490,"Kokfelt, Ulla (2018): Geochemistry of Stordale..."
1,0.007,0.007,0.074,98.3,0.073,45.7,0.7,0.034,Dicranum elongatum peat,5.0,NaN,NaN,NaN,Stordalen_1,68.35,19.05,350.0,https://doi.org/10.1594/PANGAEA.890490,"Kokfelt, Ulla (2018): Geochemistry of Stordale..."
2,0.013,0.015,0.092,98.3,0.091,48.2,0.9,0.044,Dicranum elongatum peat,5.0,NaN,NaN,NaN,Stordalen_1,68.35,19.05,350.0,https://doi.org/10.1594/PANGAEA.890490,"Kokfelt, Ulla (2018): Geochemistry of Stordale..."
3,0.018,0.022,0.091,98.4,0.090,47.3,0.7,0.043,Dicranum elongatum peat,5.0,NaN,NaN,NaN,Stordalen_1,68.35,19.05,350.0,https://doi.org/10.1594/PANGAEA.890490,"Kokfelt, Ulla (2018): Geochemistry of Stordale..."
4,0.022,0.029,0.102,98.4,0.100,47.3,0.7,0.048,Dicranum elongatum peat,5.0,NaN,NaN,NaN,Stordalen_1,68.35,19.05,350.0,https://doi.org/10.1594/PANGAEA.890490,"Kokfelt, Ulla (2018): Geochemistry of Stordale..."


download table as tab-delimited text file

In [40]:
datapath='C:/Users/kriemann/python/pangaeapy/'
outfile_sweden='Sweden_geochem.txt'

PAGES_Sweden_data.to_csv((datapath+outfile_sweden),sep='\t',index=False)

## Download many binary files

download the images from a single dataset https://doi.pangaea.de/10.1594/PANGAEA.919398

In [47]:
df_image = PanDataSet(919398)

In [48]:
df_image.data.head()

,Event,Date/Time,Content,IMAGE (Size),IMAGE (Type),IMAGE,Latitude,Longitude,Elevation
0,PS122/2_25-44,2020-02-18,Under-ice fauna in between ice platelets,1.5 MBytes,image/png,amphipodvlcsnap-2020-02-19-11h15m18s072.png,88.091158,77.685277,-4424.2
1,PS122/2_18-89,2020-01-04,Brinicles covered in platelets,1.3 MBytes,image/png,vlcsnap-2020-01-05-17h51m34s474.png,87.007125,115.393397,-4404.3
2,PS122/2_19-115,2020-01-11,"Platelet covered level ice next to """"Fort ridge""""",1015.8 kBytes,image/png,vlcsnap-2020-01-12-11h02m09s615.png,87.213587,111.324958,-4409.2
3,PS122/2_19-115,2020-01-11,"Platelet covered eastern side of """"Fort ridge""...",1.1 MBytes,image/png,vlcsnap-2020-01-12-11h03m00s535.png,87.213587,111.324958,-4409.2
4,PS122/2_19-115,2020-01-11,Platelet ice on rope,845.8 kBytes,image/png,vlcsnap-2020-01-12-11h18m23s083.png,87.213587,111.324958,-4409.2


In [49]:
df_image.data.count()

Event           62
Date/Time       62
Content         62
IMAGE (Size)    62
IMAGE (Type)    62
IMAGE           62
Latitude        62
Longitude       62
Elevation       58
dtype: int64

download files listed in the column "IMAGE"

set the prefix first (see .tab file) https://doi.pangaea.de/10.1594/PANGAEA.919398?format=textfile

In [50]:
prefix = 'https://download.pangaea.de/dataset/919398/files/'

download only images when "fauna" is listed in column "Content"

In [68]:
df_fauna = df_image.data[df_image.data['Content'].str.contains('fauna')]

In [67]:
import urllib.request 

datapath='C:/Users/kriemann/python/pangaeapy/images/'

for i, image_name in df_fauna['IMAGE'].iteritems():
    print(image_name)
    urllib.request.urlretrieve((prefix+image_name), (datapath+image_name))

amphipodvlcsnap-2020-02-19-11h15m18s072.png
vlcsnap-2020-01-26-12h55m49s566.png
vlcsnap-2020-01-26-12h56m03s753.png
vlcsnap-2020-01-26-12h57m03s914.png


## Download many files

this concerns datasets published before 2020 (in .tab file the full path is given, not just the file name)

example: https://doi.pangaea.de/10.1594/PANGAEA.910179

In [69]:
file_list = PanDataSet(910179)

In [71]:
file_list.data.head()

,File name,File format,File size,URL file,Event,Latitude,Longitude,Elevation,Date/Time
0,Cretaceous_2xPI_WMMT.nc,NC,258.055,https://store.pangaea.de/Publications/KlagesJ-...,PS104_20-2,-73.56839,-107.09153,-946.0,2017-02-21 01:21:00
1,Cretaceous_2xPI_tsurf.nc,NC,258.141,https://store.pangaea.de/Publications/KlagesJ-...,PS104_20-2,-73.56839,-107.09153,-946.0,2017-02-21 01:21:00
2,Cretaceous_4xPI_WMMT.nc,NC,257.984,https://store.pangaea.de/Publications/KlagesJ-...,PS104_20-2,-73.56839,-107.09153,-946.0,2017-02-21 01:21:00
3,Cretaceous_4xPI_tsurf.nc,NC,258.027,https://store.pangaea.de/Publications/KlagesJ-...,PS104_20-2,-73.56839,-107.09153,-946.0,2017-02-21 01:21:00
4,Cretaceous_6xPI_WMMT.nc,NC,258.039,https://store.pangaea.de/Publications/KlagesJ-...,PS104_20-2,-73.56839,-107.09153,-946.0,2017-02-21 01:21:00


In [72]:
file_list.data.count()

File name      15
File format    15
File size      15
URL file       15
Event          15
Latitude       15
Longitude      15
Elevation      15
Date/Time      15
dtype: int64

In [74]:
import urllib.request 

datapath='C:/Users/kriemann/python/pangaeapy/files/'

for i, file_url in file_list.data['URL file'].iteritems():
    print(file_url)
    urllib.request.urlretrieve(file_url, (datapath+file_list.data['File name'].iloc[i]))

https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_2xPI_WMMT.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_2xPI_tsurf.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_4xPI_WMMT.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_4xPI_tsurf.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_6xPI_WMMT.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_6xPI_tsurf.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_PI_WMMT.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/Cretaceous_PI_tsurf.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/pd_2xPI_WMMT.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/pd_2xPI_tsurf.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/pd_4xPI_WMMT.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/pd_4xPI_tsurf.nc
https://store.pangaea.de/Publications/KlagesJ-etal_2019/pd_PI_WMMT.nc
